Quick Inference with Fine-Tuned Persian QA Model

This notebook demonstrates how to load the LoRA fine-tuned Gemma-3 270M model and perform question-answering inference on new examples.

We will cover the following steps:
1.  **Setup**: Import libraries and define model paths.
2.  **Load Model & Tokenizer**: Load the base model with 4-bit quantization and apply the trained LoRA adapters.
3.  **Inference Function**: Create a helper function to format the prompt, generate an answer, and decode it.
4.  **Run Examples**: Test the model with sample questions and contexts.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import warnings

warnings.filterwarnings("ignore")

# 1. Setup Configuration

Before running, ensure the following paths are correct:
- `BASE_MODEL_NAME`: The Hugging Face identifier for the base model we fine-tuned.
- `ADAPTER_PATH`: The path to the directory where your trained LoRA adapters are saved (e.g., the `final_checkpoint` from the training script).

In [ ]:
# --- Configuration ---
BASE_MODEL_NAME = "google/gemma-3-270m"
# Adjust this path to point to your saved LoRA adapters
ADAPTER_PATH = "../outputs/final_checkpoint" 

# 2. Load Model and Tokenizer

Here, we load the base model using 4-bit quantization to reduce memory usage, which is ideal for inference on consumer GPUs. Then, we apply the fine-tuned LoRA adapters from our training process on top of it.

Finally, we call `merge_and_unload()` to combine the adapter weights with the base model weights. This creates a standard transformer model in memory, which slightly increases memory usage but significantly speeds up inference.

In [ ]:
# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Load Base Model with Quantization ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

print(f"Loading base model: {BASE_MODEL_NAME}")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": device.index} if device.type == "cuda" else "auto",
)

# --- Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- Load and Merge LoRA Adapters ---
print(f"Loading LoRA adapters from: {ADAPTER_PATH}")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

print("Merging adapters into the base model for faster inference...")
model = model.merge_and_unload()
model.eval() # Set the model to evaluation mode

print("Model and tokenizer loaded successfully.")

# 3. Inference Function

This function takes a question and a context, formats them into the prompt template used during training, and generates a short answer.

In [ ]:
def generate_response(question, context, model, tokenizer, device, max_new_tokens=64):
    """
    Generates an answer given a question and a context.
    """
    # Format the prompt exactly as it was during training
    prompt = f"پرسش: {question}\nمتن: {context}\nجواب کوتاه:"
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    
    # Generate the response
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False, # Use greedy decoding for deterministic output
        )
    
    # Decode the generated tokens, skipping the prompt part
    response = tokenizer.decode(output_tokens[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    
    return response.strip()

# 4. Run Inference Examples

Now, let's test our fine-tuned model with some examples.

In [ ]:
# --- Example 1: Chaharshanbe Suri ---

context_1 = """
چهارشنبه‌سوری یکی از جشن‌های ایرانی است که از غروب آخرین سه‌شنبهٔ ماه اسفند، تا پس از نیمه‌شب تا آخرین چهارشنبهٔ سال، برگزار می‌شود و برافروختن و پریدن از روی آتش مشخصهٔ اصلی آن است. این جشن، نخستین جشن از مجموعهٔ جشن‌ها و مناسبت‌های نوروزی است که با برافروختن آتش و برخی رفتارهای نمادین دیگر، به‌صورت جمعی در فضای باز برگزار می‌شود.
"""
question_1 = "نام جشن آخرین سه شنبه ی سال چیست؟"

print("--- Example 1 ---")
print(f"Context: {context_1.strip()}")
print(f"Question: {question_1}")

answer_1 = generate_response(question_1, context_1, model, tokenizer, device)
print(f"\nGenerated Answer: {answer_1}")
print("-" * 20)

In [ ]:
# --- Example 2: The Good, the Bad and the Ugly ---

context_2 = """
خوب، بد، زشت یک فیلم در ژانر وسترن اسپاگتی حماسی است که توسط سرجو لئونه در سال ۱۹۶۶ در ایتالیا ساخته شد. زبانی که بازیگران این فیلم به آن تکلم می‌کنند مخلوطی از ایتالیایی و انگلیسی است. این فیلم سومین (و آخرین) فیلم از سه‌گانهٔ دلار (Dollars Trilogy) و در حال حاضر در فهرست ۲۵۰ فیلم برتر تاریخ سینمای جهان شناخته می‌شود. در فیلم، با نام «بلوندی» و «زشت» (ایلای والک، در فیلم، با نام «توکو») با هم کار می‌کنند و با شگرد خاصی، به گول زدن کلانترهای مناطق مختلف و پول درآوردن از این راه می‌پردازند. «بد» (لی وان کلیف) آدمکشی حرفه‌ای است که به خاطر پول حاضر به انجام هر کاری است.
"""
question_2 = "شخصیت بد در فیلم خوب، بد، زشت چه کسی بود؟"

print("\n--- Example 2 ---")
print(f"Context: {context_2.strip()}")
print(f"Question: {question_2}")

answer_2 = generate_response(question_2, context_2, model, tokenizer, device)
print(f"\nGenerated Answer: {answer_2}")
print("-" * 20)

In [ ]:
# --- Example 3: Crescent Petroleum Contract ---
context_3 = """
قرارداد کرسنت قراردادی برای فروش روزانه معادل ۵۰۰ میلیون فوت مکعب، گاز ترش میدان سلمان است، که در سال ۱۳۸۱ و در زمان وزارت بیژن نامدار زنگنه در دولت هفتم مابین شرکت کرسنت پترولیوم و شرکت ملی نفت ایران منعقد گردید. مذاکرات اولیه این قرارداد از سال ۱۹۹۷ آغاز شد و در نهایت، سال ۲۰۰۱ (۱۳۸۱) به امضای این تفاهم نامه مشترک انجامید.
"""
question_3 = "قرارداد کرسنت در چه سالی منعقد شد؟"

print("\n--- Example 3 ---")
print(f"Context: {context_3.strip()}")
print(f"Question: {question_3}")

answer_3 = generate_response(question_3, context_3, model, tokenizer, device)
print(f"\nGenerated Answer: {answer_3}")
print("-" * 20)